# Анализ базы данных сервиса для чтения книг по подписке

Задача - проанализировать базу данных с информацией о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

Импортируем основные библиотеки.

In [2]:
db_config = {'user': 'praktikum_student',
 'pwd': 'Sdf4$2;d-d30pp',
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, 
 'db': 'data-analyst-final-project-db'} 

Установим параметры.

In [3]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db']) 

Пропишем строку соединения

In [4]:
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

Сохраним строку соединения

In [5]:
books = ''' SELECT *
            FROM books
        '''

Запишем текст запроса к таблице books

In [6]:
df_books=pd.io.sql.read_sql(books, con = engine)
df_books.sample(3)

,book_id,author_id,title,num_pages,publication_date,publisher_id
643,644,529,The Birth of Venus,427,2004-11-30,237
762,763,211,The Invisible Man,192,2002-09-03,267
635,636,561,The Art of War,273,2005-01-11,264


Запишем таблицу books в датафрейм. Выведем 3 случайных строки датафрейма

In [7]:
df_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
book_id             1000 non-null int64
author_id           1000 non-null int64
title               1000 non-null object
num_pages           1000 non-null int64
publication_date    1000 non-null object
publisher_id        1000 non-null int64
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


Посмотрим на информацию о таблице

In [8]:
authors = ''' SELECT *
            FROM authors
        '''

In [9]:
df_authors=pd.io.sql.read_sql(authors, con = engine)
df_authors.sample(3)

,author_id,author
420,421,Mitch Albom
317,318,Jonathan Swift/Robert DeMaria Jr.
133,134,Dodie Smith


In [10]:
df_authors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
author_id    636 non-null int64
author       636 non-null object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


Сделаем аналогичные шаги с таблицей authors

In [11]:
publishers = ''' SELECT *
            FROM publishers
        '''

In [12]:
df_publishers=pd.io.sql.read_sql(publishers, con = engine)
df_publishers.sample(3)

,publisher_id,publisher
171,172,Laurel Leaf
252,253,Scholastic
100,101,Ember


In [13]:
df_publishers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
publisher_id    340 non-null int64
publisher       340 non-null object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


Выведем информацию о таблице с издательствами

In [14]:
ratings = ''' SELECT *
            FROM ratings
        '''

In [15]:
df_ratings = pd.io.sql.read_sql(ratings, con = engine)
df_ratings.sample(3)

,rating_id,book_id,username,rating
5475,5476,847,dwayne09,3
4108,4109,670,odavila,3
2920,2921,461,fnguyen,5


In [16]:
len(df_ratings['book_id'].unique())

1000

In [17]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
rating_id    6456 non-null int64
book_id      6456 non-null int64
username     6456 non-null object
rating       6456 non-null int64
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


Прочтем таблицу с пользовательскими рейтингами

In [18]:
reviews = ''' SELECT *
            FROM reviews
        '''

In [19]:
df_reviews = pd.io.sql.read_sql(reviews, con = engine)
df_reviews.head(3)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...


In [20]:
len(df_reviews['book_id'].unique())

994

В датафрейме - отзывы о 994 книгах из 1000, не хватает отзывов о 6 книгах.

In [21]:
df_missing_reviews = df_books.merge(df_reviews, on = 'book_id', how = 'left')
df_missing_reviews.query('review_id.isna()')

,book_id,author_id,title,num_pages,publication_date,publisher_id,review_id,username,text
241,83,174,Anne Rice's The Vampire Lestat: A Graphic Novel,404,1991-11-30,33,NaN,NaN,NaN
520,191,568,Disney's Beauty and the Beast (A Little Golden...,24,2004-05-11,113,NaN,NaN,NaN
605,221,151,Essential Tales and Poems,688,2004-10-25,42,NaN,NaN,NaN
1055,387,365,Leonardo's Notebooks,352,2005-08-01,49,NaN,NaN,NaN
1854,672,144,The Cat in the Hat and Other Dr. Seuss Favorites,61,2003-10-14,174,NaN,NaN,NaN
2249,808,350,The Natural Way to Draw,240,1990-02-01,178,NaN,NaN,NaN


Выведем книги, о которых нет отзывов.

In [22]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
review_id    2793 non-null int64
book_id      2793 non-null int64
username     2793 non-null object
text         2793 non-null object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


In [23]:
df_reviews.duplicated().sum()

0

In [24]:
df_reviews['book_id'].describe()

count    2793.000000
mean      504.693161
std       288.472931
min         1.000000
25%       259.000000
50%       505.000000
75%       753.000000
max      1000.000000
Name: book_id, dtype: float64

In [25]:
cols = df_reviews.columns[:6]
for col in df_reviews.columns:
    pct_missing = np.mean(df_reviews[col].isna())
    print('{} - {}%'.format(col, pct_missing*100))

review_id - 0.0%
book_id - 0.0%
username - 0.0%
text - 0.0%


Прочтем таблицу с ревью

In [26]:
books_after_1_jan_2000 = '''select count(book_id)
                            from books
                            where cast(publication_date as date)>='2000-01-01'
                            '''

In [27]:
books_after_1_jan_2000_qty = pd.io.sql.read_sql(books_after_1_jan_2000, con = engine)
books_after_1_jan_2000_qty

,count
0,821


Посчитаем, сколько книг вышло после 1 января 2000 года. После этой даты вышла 821 книга.

In [28]:
reviews_and_ratings = '''select subq.title, subq.average_raiting, count(reviews.username) as review_qty
                        from
                            (select books.title as title, avg(ratings.rating) as average_raiting, books.book_id as book_id
                            from books
                            left join ratings on ratings.book_id = books.book_id
                            group by books.title, books.book_id) as subq
                            left join reviews on reviews.book_id = subq.book_id
                        group by subq.title, subq.average_raiting
                        order by average_raiting desc, review_qty desc'''

In [29]:
reviews_and_ratings_table = pd.io.sql.read_sql(reviews_and_ratings, con = engine)
reviews_and_ratings_table

,title,average_raiting,review_qty
0,A Dirty Job (Grim Reaper #1),5.00,4
1,Moneyball: The Art of Winning an Unfair Game,5.00,3
2,School's Out—Forever (Maximum Ride #2),5.00,3
3,A Woman of Substance (Emma Harte Saga #1),5.00,2
4,Tai-Pan (Asian Saga #2),5.00,2
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,2.25,3
996,Drowning Ruth,2.00,3
997,His Excellency: George Washington,2.00,2
998,Junky,2.00,2


In [30]:
reviews_and_ratings_table['review_qty'].sum()

2793

In [31]:
reviews_and_ratings_table['average_raiting'].describe()

count    1000.000000
mean        3.898973
std         0.562376
min         1.500000
25%         3.500000
50%         4.000000
75%         4.333333
max         5.000000
Name: average_raiting, dtype: float64

In [32]:
reviews_and_ratings_table['review_qty'].describe()

count    1000.000000
mean        2.793000
std         1.074852
min         0.000000
25%         2.000000
50%         3.000000
75%         3.000000
max         7.000000
Name: review_qty, dtype: float64

Для каждой книги посчитаем количество обзоров и среднюю оценку

In [33]:
fatest_publisher = ''' select max(subq.count), subq.publisher
                        from                        
                            (select publishers.publisher, count(books.book_id)
                            from
                                books
                                left join publishers on publishers.publisher_id = books.publisher_id
                            where books.num_pages>50
                            group by
                            publishers.publisher
                            order by count desc) AS subq
                        group by subq.publisher
                        order by max desc
                    '''

In [34]:
fatest_publisher_table = pd.io.sql.read_sql(fatest_publisher, con = engine)
fatest_publisher_table

,max,publisher
0,42,Penguin Books
1,31,Vintage
2,25,Grand Central Publishing
3,24,Penguin Classics
4,19,Ballantine Books
...,...,...
329,1,Noonday Press
330,1,Atheneum Books for Young Readers: Richard Jack...
331,1,Penguin Signet
332,1,Victor Gollancz


Определим издательство, выпустившее максимальное количество книг толще 50 страниц (чтобы не учитывать брошюры). Это Penguin Books

In [35]:
best_author = '''   select authors.author, avg(ratings.rating) as avg_raiting
                    from books
                    left join authors on authors.author_id = books.author_id
                    left join ratings on ratings.book_id = books.book_id
                    where books.book_id in  
                        (select ratings.book_id
                        from ratings
                        group by ratings.book_id
                        having(count(ratings.book_id)>50))
                    group by authors.author
                    order by avg_raiting desc
                '''

In [36]:
best_author_table = pd.io.sql.read_sql(best_author, con = engine)
best_author_table

,author,avg_raiting
0,J.K. Rowling/Mary GrandPré,4.287097
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.246914
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Lois Lowry,3.750000


Определите автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками. Это J.K. Rowling, средняя оценка 4,287

In [37]:
avg_score = ''' 
                    select avg(cnt)
                    from
                    (select reviews.username as name, count(reviews.username) as cnt
                    from reviews
                    where reviews.username in
                        (select ratings.username
                        from ratings
                        group by ratings.username
                        having(count(ratings.book_id)>50))
                    group by reviews.username) as subq
                '''

In [38]:
avg_score_table = pd.io.sql.read_sql(avg_score, con = engine)
avg_score_table

,avg
0,24.333333


Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок. В среднем такие пользователи делают 24,3 обзора.

# Общие выводы

- В базе содержится информация о 1000 книг.

- В процессе исследовательского анализа данных было выявлено, что отсутствуют отзывы о 6 книгах.

- После 1 января 2000 года вышла 821 книга.

- В датафрейме reviews_and_ratings_table представлены данные о рейтингах на каждую книгу и об отзывах на 994 книги из 1000. Средняя оценка - 3,98 балла из 5. Медианная оценка - 4 балла. На каждую книгу в среднем есть 3 отзыва. Максимальное количество отзывов на книгу - 7.

- Издательство, выпустившее максимальное количество книг толще 50 страниц - Penguin Books.

- Автор с самой высокой средней оценкой книг(учитывая только книги с 50 и более оценками) - J.K. Rowling, средняя оценка 4,287 балла.

- Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24,3 обзора.